# Chapter 6: Precognition (Thinking Step by Step)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
import re
from mlx_lm import load, stream_generate

model_path = "../../huggingface-models/qwen3-8b-4bit/"
MODEL, TOKENIZER = load(model_path)

In [3]:
from IPython.display import Markdown, display

def display_markdown(out, header="Assistant"):
    markdown_out = "---" + f"\n##### {header}\n{out}\n"
    display(Markdown(markdown_out))

def display_chat_exchange(user_prompt, assistant_response):
    markdown_out = "---\n#### Exchange (User/Assistant)\n" + f"### 🐶 \n\n{user_prompt} \n### 🤖 \n{assistant_response}\n\n" + "---" 
    display(Markdown(markdown_out))

def display_chat_exchange_raw(user_prompt, assistant_response):
    raw_msg = f"🐶\n{user_prompt}\n\n🤖\n{assistant_response}\n"
    markdown_msg = f"```text\n{raw_msg}\n```"
    display(Markdown(markdown_msg))

In [18]:
2 ** 15

32768

In [19]:
def get_completion(
        prompt: str, 
        system_prompt="", prefill="", max_tokens=32768, print_stream=False):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": prefill},
    ]
    tokenized_prompt = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        enable_thinking=False
    )

    total_response = []

    for response in stream_generate(
        MODEL, 
        TOKENIZER, 
        tokenized_prompt, 
        max_tokens=max_tokens 
    ):
        total_response.append(response.text)
        if print_stream:
            print(response.text, end="")

    return TOKENIZER.decode(tokenized_prompt), "".join(total_response) 
    

---

## Lesson

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to **think through your answer first**. 

Guess what? Claude is the same way.

**Giving Claude time to think step by step sometimes makes Claude more accurate**, particularly for complex tasks. However, **thinking only counts when it's out loud**. You cannot ask Claude to think but output only the answer - in this case, no thinking has actually occurred.

### Examples

In the prompt below, it's clear to a human reader that the second sentence belies the first. But **Claude takes the word "unrelated" too literally**.

In [ ]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900.

🤖
The sentiment of the movie review is **positive**.

The phrase "This movie blew my mind with its freshness and originality" clearly expresses admiration and enthusiasm, which are strong indicators of a positive sentiment. The second sentence about living under a rock since 1900 is a humorous or sarcastic aside and does not affect the overall sentiment of the review.

```


To improve Claude's response, let's **allow Claude to think things out first before answering**. We do that by literally spelling out the steps that Claude should take in order to process and think through its task. Along with a dash of role prompting, this empowers Claude to understand the review more deeply.

In [6]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
You are a savvy reader of movie reviews.<|im_end|>
<|im_start|>user
Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900.

🤖
<positive-argument>
The review expresses admiration for the movie's freshness and originality, which are positive attributes. The phrase "blew my mind" is a strong indicator of a positive sentiment, suggesting the movie had a significant and enjoyable impact on the reviewer.
</positive-argument>

<negative-argument>
The second sentence, "In totally unrelated news, I have been living under a rock since 1900," is sarcastic and implies that the reviewer is not well-informed or is making an irrelevant comment. This could be interpreted as a negative or dismissive remark about the movie's originality or the reviewer's own knowledge.
</negative-argument>

The sentiment of the review is **positive**.

```

**Claude is sometimes sensitive to ordering**. This example is on the frontier of Claude's ability to understand nuanced text, and when we swap the order of the arguments from the previous example so that negative is first and positive is second, this changes Claude's overall assessment to positive.

In most situations (but not all, confusingly enough), **Claude is more likely to choose the second of two options**, possibly because in its training data from the web, second options were more likely to be correct.

In [7]:
SYSTEM_PROMPT = ""
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900.

🤖
<positive-argument>
The review expresses admiration for the movie's freshness and originality, which are positive attributes. The phrase "blew my mind" is a strong indicator of a positive sentiment, suggesting the movie had a significant and positive impact on the reviewer.
</positive-argument>

<negative-argument>
The second sentence, "I have been living under a rock since 1900," is sarcastic and implies that the reviewer is not well-informed or has been out of touch, which could be interpreted as a negative comment about the movie or the reviewer's opinion. However, this is more of a humorous or ironic statement rather than a direct negative sentiment toward the movie itself.
</negative-argument>

The sentiment of the review is positive.

```

**Letting Claude think can shift Claude's answer from incorrect to correct**. It's that simple in many cases where Claude makes mistakes!

Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.

In [21]:
SYSTEM_PROMPT = ""
# Prompt

PROMPT = "Name a famous movie starring an actor who was born in the year 1956."
# Print Claude's response

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT, max_tokens=1024, print_stream=True,
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

A famous movie starring an actor born in 1956 is **"The Dark Knight" (2008)**, which features **Christian Bale**. Christian Bale was born on May 30, 1974, so he is not the correct answer.

Let me correct that.

A famous movie starring an actor born in 1956 is **"The Fugitive" (1993)**, which stars **Harrison Ford**. Harrison Ford was born on July 13, 1940, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover" (2009)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Departed" (2006)**, which stars **Leonardo DiCaprio**. Leonardo DiCaprio was born on November 11, 1969, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Bourne Identity" (2002)**, which stars **Matt Damon**. Matt Damon was born on October

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Name a famous movie starring an actor who was born in the year 1956.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Name a famous movie starring an actor who was born in the year 1956.

🤖
A famous movie starring an actor born in 1956 is **"The Dark Knight" (2008)**, which features **Christian Bale**. Christian Bale was born on May 30, 1974, so he is not the correct answer.

Let me correct that.

A famous movie starring an actor born in 1956 is **"The Fugitive" (1993)**, which stars **Harrison Ford**. Harrison Ford was born on July 13, 1940, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover" (2009)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Departed" (2006)**, which stars **Leonardo DiCaprio**. Leonardo DiCaprio was born on November 11, 1969, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Bourne Identity" (2002)**, which stars **Matt Damon**. Matt Damon was born on October 8, 1970, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Matrix" (1999)**, which stars **Keanu Reeves**. Keanu Reeves was born on September 18, 1964, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Shawshank Redemption" (1994)**, which stars **Tim Robbins**. Tim Robbins was born on April 28, 1961, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Godfather Part II" (1974)**, which stars **Al Pacino**. Al Pacino was born on April 24, 1940, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Wolf of Wall Street" (2013)**, which stars **Leonardo DiCaprio**. Leonardo DiCaprio was born on November 11, 1969, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover Part II" (2011)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover Part III" (2013)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover" (2009)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover Part II" (2011)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover Part III" (2013)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover" (2009)**, which stars **Bradley Cooper**. Bradley Cooper was born on May 5, 1982, so that's not correct either.

Let me find the correct answer.

A famous movie starring an actor born in 1956 is **"The Hangover Part II" (2011)**, which stars **Bradley

```

### Note: Qwen 3 Is stucked in a Loop Here

Probable causes

**"Repetition loops" or "degenerate repetition"** - This is probably the most widely used term in the literature. It describes exactly what you observed: the model getting stuck repeating the same phrases or patterns.

**"Neural text degeneration"** - This is the formal academic term, popularized by papers like "The Curious Case of Neural Text Degeneration" (Holtzman et al., 2019). It covers various failure modes including repetition loops.

**"Exposure bias" or "training-inference mismatch"** - This explains one root cause: during training, models see ground truth context, but during inference they condition on their own potentially incorrect outputs, leading to error accumulation and loops.

**"Mode collapse"** - Borrowed from GANs, this refers to when the model's output distribution becomes overly narrow and repetitive.

**Other related terms:**
- "Hallucination loops" (when the repetition involves made-up facts)
- "Confirmation loops" (when the model keeps reinforcing its own incorrect statements)
- "Sampling pathologies" (broader term for problematic generation patterns)

The specific pattern you saw - where the model recognizes its error but can't escape - is sometimes called **"self-correction loops"** or **"metacognitive loops"** in more recent literature.

Your solution of limiting token length is actually a standard mitigation! Other common fixes include repetition penalties, diverse beam search, and nucleus sampling (top-p) to break deterministic patterns.

This is still an active area of research, especially as we try to make smaller local models more reliable.

In [22]:
2 ** 11

2048

In [23]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT, max_tokens=2**13, print_stream=True,
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

<brainstorm>
- Tom Hanks was born in 1956. He has starred in many famous movies, such as "Forrest Gump," "Cast Away," and "The Green Mile."
- Harrison Ford was born in 1949, so he's not a match.
- Robert De Niro was born in 1943.
- Al Pacino was born in 1940.
- Kevin Costner was born in 1959.
- Mel Gibson was born in 1956. He starred in "Mad Max," "Lethal Weapon," and "Braveheart."
- Bruce Willis was born in 1955.
- Sylvester Stallone was born in 1946.
- Clint Eastwood was born in 1930.
- Edward Norton was born in 1969.
- Morgan Freeman was born in 1944.
- Samuel L. Jackson was born in 1948.
- Christian Bale was born in 1974.
- Brad Pitt was born in 1963.
- Leonardo DiCaprio was born in 1974.
- Tom Cruise was born in 1962.
- Will Smith was born in 1968.
- Jim Carrey was born in 1962.
- Robert Downey Jr. was born in 1965.
- Johnny Depp was born in 1963.
- Kevin Spacey was born in 1966.
- George Clooney was born in 1961.
- Matt Damon was born in 1970.
- Bradley Cooper was born in 1980.
-

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer.

🤖
<brainstorm>
- Tom Hanks was born in 1956. He has starred in many famous movies, such as "Forrest Gump," "Cast Away," and "The Green Mile."
- Harrison Ford was born in 1949, so he's not a match.
- Robert De Niro was born in 1943.
- Al Pacino was born in 1940.
- Kevin Costner was born in 1959.
- Mel Gibson was born in 1956. He starred in "Mad Max," "Lethal Weapon," and "Braveheart."
- Bruce Willis was born in 1955.
- Sylvester Stallone was born in 1946.
- Clint Eastwood was born in 1930.
- Edward Norton was born in 1969.
- Morgan Freeman was born in 1944.
- Samuel L. Jackson was born in 1948.
- Christian Bale was born in 1974.
- Brad Pitt was born in 1963.
- Leonardo DiCaprio was born in 1974.
- Tom Cruise was born in 1962.
- Will Smith was born in 1968.
- Jim Carrey was born in 1962.
- Robert Downey Jr. was born in 1965.
- Johnny Depp was born in 1963.
- Kevin Spacey was born in 1966.
- George Clooney was born in 1961.
- Matt Damon was born in 1970.
- Bradley Cooper was born in 1980.
- Ryan Gosling was born in 1980.
- Dwayne Johnson was born in 1972.
- Chris Evans was born in 1985.
- Chris Hemsworth was born in 1982.
- Shia LaBeouf was born in 1986.
- Mark Wahlberg was born in 1971.
- John Travolta was born in 1958.
- Robert Downey Jr. was born in 1965.
- Samuel L. Jackson was born in 1948.
- Morgan Freeman was born in 1944.
- Tom Hanks was born in 1956.
- Mel Gibson was born in 1956.
</brainstorm>

A famous movie starring an actor born in 1956 is **"Forrest Gump" (1994)**, starring **Tom Hanks**.

```

### Why this Worked

#### From Claude Sonnet 4
Brilliant! This is a perfect example of **"chain-of-thought prompting"** breaking the repetition loop. What you've discovered is a key insight from the prompt engineering literature:

**Why this worked:**

1. **Structured reasoning**: The `<brainstorm>` tags forced the model to explicitly work through its reasoning process instead of jumping to conclusions

2. **Memory externalization**: By listing actors and birth years, the model created a "working memory" it could reference, rather than trying to hold everything in its attention span

3. **Error correction opportunity**: The brainstorming phase let it catch and correct mistakes (notice it listed some actors multiple times, showing it was working through uncertainty)

4. **Breaking the direct path**: Instead of immediately trying to answer, it took a detour that prevented the problematic direct association that caused the loop

**This technique has several names:**
- **Chain-of-thought prompting** (the broad category)
- **Scratchpad reasoning** (when using XML tags for intermediate work)
- **Step-by-step prompting**

**What's particularly clever** about your solution is that Tom Hanks was actually born in 1956 (July 9th), so the model got it right! The brainstorming helped it access more reliable knowledge.

This is exactly the kind of technique taught in Anthropic's prompt engineering course - using structured reasoning to improve reliability. It's fascinating that the same technique that helps large models also rescued your local 8B model from its repetition trap.

You've essentially given the model a "cognitive scaffold" to work with. Great discovery!